# Text mining jaarverslagen
1. Read info from the `file_list.csv`.
2. Uses `analyse_jaarverslag.ipynb` to analyse each jaarverslag, one at a time.

## 1. Read from `file_list.csv`

In [19]:
from os.path import join, isfile
import pandas as pd

folder = './jaarverslagen'
files = pd.read_csv(join(folder, 'file_list.csv'))
files

,filename,language
0,ABNAMRO_2017.pdf,english
1,AEGON_2017.pdf,english
2,Akzonobel_2017.pdf,english
3,Heineken_2017.pdf,english
4,ING_Groep_2017.pdf,english
5,KPN_2017.pdf,english
6,Philips_2017.pdf,english
7,Unilever_2017.pdf,english


## 2. Run through all jaarverslagen
Using `papermill`, see https://papermill.readthedocs.io/en/latest/usage.html.

How does this work with my virtual environment `tmj`?
* Jupyter Notebook is served from `base` environment which has `widgetnbextension` enabled
* Next `ipywidgets` is installed in this kernel's environment `tmj`
* See https://ipywidgets.readthedocs.io/en/stable/user_install.html#installing-with-multiple-environments for more info about this

In [20]:
import papermill as pm

We'll execute the notebook `analyse_jaarverslag` and have the same output file as the input file. This means that the parameters will be inserted (`injected-paramaters` cell) into the original notebook.

In [23]:
# Loop over all files
for row in files.itertuples():
    
    #
    print('Running:', row.filename)
    
    # First check if it exists
    if not isfile(join(folder, row.filename)):
        print('File not found:', row.filename)
        continue
    
    # Execute the notebook
    pm.execute_notebook(
       'analyse_jaarverslag.ipynb',
       'analyse_jaarverslag.ipynb',
       parameters = dict(filename = row.filename,
                         folder = folder,
                         language = row.language)
    )

Running: ABNAMRO_2017.pdf



Running: AEGON_2017.pdf



Running: Akzonobel_2017.pdf



Running: Heineken_2017.pdf



Running: ING_Groep_2017.pdf



Running: KPN_2017.pdf



Running: Philips_2017.pdf



Running: Unilever_2017.pdf


## 3. Extract the data from the output folders
And combine it of course.

## 4. Inverse document frequency